# TP 1 : Clustering et k-means

alexandre.bruckert@univ-nantes.fr

Ce TP est librement inspiré du [Python Data Science Handbook](https://www.oreilly.com/library/view/python-data-science/9781491912126/) de Jake VanderPlas.

Dans ce TP, nous allons implémenter l'algorithme du k-means (et quelques variantes), et l'appliquer à divers types de données.

Tout d'abord, importons quelques librairies utiles 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import random
from sklearn.datasets import make_blobs
from sklearn.datasets import load_sample_image
from PIL import Image

Puis nous allons générer un premier ensemble de données synthétiques

In [ ]:
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.70)
# Même si la fonction make_blobs génère des données labélisées y_true, on ne va pas les utiliser ici
plt.scatter(X[:, 0], X[:, 1], s=50);

On va maintenant implémenter l'algorithme de Lloyd du k-means

In [ ]:
def kmeans(X, k):
    """
    Implémentation de l'algorithme du k-means
    @inputs :
        - X: numpy array, de taille n_samples * 2, coordonnées des points 
        - k: int, nombre de clusters
    @outputs :
        - centers: numpy array, position des centroïdes
        - cluster_labels: numpy array, cluster assigné pour chaque point de X
    """
    # Assigner les centroïdes des cluster (choisir k points aléatoires de X)
    centers = # TODO
    
    while True:
        # Calculer la distance entre chaque point de X et les différents centroïdes, 
        # et assigner chaque point au centroïde le plus proche
        # Hint : aller voir la fonction pairwise_distances_argmin de sklearn.metrics
        cluster_labels = # TODO
        
        # Calculer la position des nouveaux centroïdes
        new_centers = # TODO
        
        # Vérifier la convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, cluster_labels

Visualisons les clusters obtenus

In [ ]:
centers, cluster_labels = kmeans(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=cluster_labels, s=50, cmap='viridis');

Parfois, la solution n'est pas l'optimum global; faites tourner l'algorithme plusieurs fois, en observant à chaque fois les clusters générés

In [ ]:
# TODO

Le nombre de clusters choisi a également un impact fort ! Faites tourner à nouveau l'algorithme, en faisant varier le nombre de clusters k

In [ ]:
# TODO

Comme indiqué en cours, plusieurs méthodes existent pour estimer le nombre "optimal" de clusters.

Commençons tout d'abord par générer un nouveaux jeu de données, avec cette fois ci un nombre aléatoire de cluster

In [ ]:
n_centers = random.sample(range(1,10),1)[0]
X, y_true = make_blobs(n_samples=1000, centers=n_centers, cluster_std=0.70)
plt.scatter(X[:, 0], X[:, 1], s=50);

Implémentons maintenant une fonction d'évaluation, pour mesurer la distance moyenne de chaque point à son centroïde

In [ ]:
def average_distance_to_centroid(X, centers, cluster_labels):
    # TODO
    # Hint : utiliser la norme euclidienne Numpy np.linalg.norm
    return avg_dist

Maintenant, implémenter une fonction qui appliquera l'algorithme du k-means en augmentant progressivement le nombre de clusters, puis affichez l'évolution de la distance moyenne aux centroïdes.

In [ ]:
def iterate_kmeans(X, k_min, k_max):
    # TODO
    return avg_dist_array

# TODO : afficher la courbe montrant l'évolution de la distance moyenne aux centroïdes

Que pouvez-vous en déduire sur le nombre initial n_centers de centroïdes dans les données générées ? Vérifiez votre hypothèse !

In [ ]:
# TODO

Bonus : en utilisant [l'exemple suivant de la librairie scikit-learn](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html), calculez le coefficient de silhouette pour différents nombre de clusters. Affichez les courbes de silhouettes, et validez votre hypothèse sur le nombre optimal de clusters.

In [ ]:
# TODO

### Application : Compression de couleur d'image

Dans cette partie, nous allons appliquer l'algorithme du k-means à une tâche particulière: la compression de couleurs dans une image.

Dans beaucoup de formats, les images sont stockées comme 3 matrices prenant des valeurs entre 0 et 255 (i.e. 8-bits): une pour le canal rouge, une pour le bleu, et une pour le vert. Une image encodée de la sorte a donc un ensemble de couleurs possibles de 256^3 = ~17 millions. Or, dans la plupart des cas, la majorité de ces couleurs ne sera pas utilisée, et beaucoup de pixels de l'image auront des couleurs très proches, voire identiques.

Nous allons donc utiliser le clustering pour assigner à chaque pixel une couleur unique, parmi une présélection limitée de couleurs, afin de réduire le poids de l'image.

In [ ]:
img = Image.open("peppers.jpg")
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(img);

On vérifie maintenant la forme et la taille de l'image

In [ ]:
img = np.array(img)
print(img.shape)

Maintenant, nous allons transformer l'image de telle sorte que les valeurs soient dans \[0, 1\], et modifier sa forme de sorte à obtenir un vecteur de $R^3$

In [ ]:
data = img / 255.
data = data.reshape(img.shape[0] * img.shape[1], 3)
print(data.shape)

On peut commencer par observer les distributions des pixels dans l'espace des couleurs

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20);

In [ ]:
plot_pixels(data, title='Input color space: 17 million possible colors')

Maintenant, on va appliquer un clustering en utilisant notre algorithme de k-means sur l'ensemble des pixels. On va donc réduire le nombre de couleurs possibles à k.

In [ ]:
# TODO 
new_colors = # TODO

plot_pixels(data, colors=new_colors, title="Reduced color space: 16 colors")

Maintenant, reformons l'image initiale

In [ ]:
img_recolored = new_colors.reshape(img.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(img)
ax[0].set_title('Original Image')
ax[1].imshow(img_recolored)
ax[1].set_title('16-color Image');

Vous pouvez essayer de faire varier les images, le nombre de couleurs, etc...

Bonus : Vous avez sans doute remarqué que l'algorithme du k-means a mis un peu de temps à tourner dans le cas de l'image. Essayez avec une image plus grande (haute résolution), et comparez le temps de calcul.

Utilisez maintenant la fonction [MiniBatchKmeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) de sklearn.cluster, et comparez les temps de calcul.

In [ ]:
# TODO